In [7]:
import os
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
files = os.listdir("data")

In [9]:
files

['geschaeft2007_496.htm',
 'geschaeft1997_538.htm',
 'geschaeft2015_90.htm',
 'geschaeft2000_617.htm',
 'geschaeft2000_165.htm',
 'geschaeft1998_185.htm',
 'geschaeft2002_289.htm',
 'geschaeft2000_159.htm',
 'geschaeft2007_455.htm',
 'geschaeft2002_262.htm',
 'geschaeft1997_289.htm',
 'geschaeft1999_433.htm',
 'geschaeft1998_146.htm',
 'geschaeft2000_415.htm',
 'geschaeft2000_367.htm',
 'geschaeft2010_429.htm',
 'geschaeft2006_444.htm',
 'geschaeft2003_501.htm',
 'geschaeft1998_436.htm',
 'geschaeft2002_63.htm',
 'geschaeft1998_350.htm',
 'geschaeft2005_356.htm',
 'geschaeft2009_172.htm',
 'geschaeft2000_207.htm',
 'geschaeft2005_430.htm',
 'geschaeft2008_375.htm',
 'geschaeft2009_166.htm',
 'geschaeft2004_179.htm',
 'geschaeft2000_549.htm',
 'geschaeft2003_307.htm',
 'geschaeft2004_421.htm',
 'geschaeft2000_7.htm',
 'geschaeft2019_84.htm',
 'geschaeft1999_584.htm',
 'geschaeft1997_470.htm',
 'geschaeft2004_409.htm',
 'geschaeft2012_464.htm',
 'geschaeft2017_56.htm',
 'geschaeft2005_19

In [459]:
file = open("data/" + str(files[0]), "r")

In [450]:
text = file.read()

In [451]:
soup = BeautifulSoup(text, "html.parser")

In [452]:
soup

<html lang="de"><head><meta content="text/html;charset=utf-8" http-equiv="Content-type"/><meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/><meta content="Gemeinderat Zürich" name="author"/>
<meta content="" name="description"/>
<meta content="" name="keywords"/>
<title>
	Detailansicht Geschäft | Gemeinderat Zürich
</title><link href="/css/Normalize.css" rel="stylesheet"/><link href="/css/grz-theme/jquery-ui-1.10.4.custom.css" rel="stylesheet"/><link href="/css/GRZ.css" rel="stylesheet"/><link href="/css/lightbox.css" rel="stylesheet"/><link href="/css/Gemeinderat.css" rel="stylesheet"/><link href="/css/colorbox.css" rel="stylesheet"/><link href="/scripts/socialprivacy/stylesheets/socialshareprivacy.css" rel="stylesheet"/>
<script src="/scripts/jquery-1.11.0.js" type="text/javascript"></script>
<script src="/scripts/jquery-ui-1.10.4.custom.js" type="text/javascript"></script>
<script src="/scripts/jquery.colorbox-min.js" type="text/javascript"></script>
<script src="/script

### Schritt 1: Die Name und Partei lokalisieren

In [453]:
#Hierist der Erstunterzeichner drin:
soup.find_all("a")[23]

<a class="unterzeichnenderLink" href="/Mitglieder/Detailansicht-Mitglied?mid=3ad30c09-6873-4ee0-a9cf-8b699a5e943c" id="ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_GeschaeftControl_13_ErstUnterzeichnender">Bernhard im Oberdorf (SVP) </a>

In [475]:
#Prüfen wir das an einem anderen Beispiel mit zwei Unterzeichnenden:
file2 = open("data/geschaeft1999_433.htm", "r")
text2 = file2.read()
soup2 = BeautifulSoup(text2, "html.parser")
soup2.find_all("a")[23:25]

#bei mehreren ist das in Positionen 23,24 etc. drin > Problem: das ist nicht stabil, Positionen verschieben sich
# stattdessen mit div_Tag probeieren

[<a class="unterzeichnenderLink" href="/Mitglieder/Detailansicht-Mitglied?mid=49d8a461-cfec-48dd-b863-15a6f471b1a8" id="ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_GeschaeftControl_13_ErstUnterzeichnender">Christian Mettler (SVP) </a>,
 <a class="pdfLinkIcon" href="/Geschaefte/detailansicht-geschaeft/Dokument/40e888e4-00dd-4582-8710-7adbaba9ac23/1999_0433.pdf" target="_blank">1999_0433.pdf (59 KB)</a>]

In [476]:
#Fazit: es die Namen sind alle in Div-Tag Nr. 24, und dort als a-Tags erfasst:
soup2.find_all("div")[24].find_all("a")

#Und so holt man sie raus:
soup2.find_all("div")[24].find_all("a")[0].text.strip()

'Christian Mettler (SVP)'

In [477]:
# und das jetzt noch splitten: 
partei = soup2.find_all("div")[24].find_all("a")[0].text.strip().split(" (")[1][:-1]
partei

name = soup2.find_all("div")[24].find_all("a")[0].text.strip().split(" (")[0]
name
vorname = name.split(" ")[0]
vorname

'Christian'

### Schritt 2: Geschäftsnummer rauslesen

In [478]:
soup2.find_all("div")[22].find_all("span")[1].text

'1999/433'

### Schritt 3: Departement rauslesen

In [479]:
soup2.find_all("div")[28].find_all("span")[2].text

IndexError: list index out of range

In [480]:
#funktioniert nicht immer, deshlab vielleicht eher mit Regex

In [481]:
import re

try:
    regex_dep = r"((?<=Zuständiges Departement).*\n*.*)"
    dep_fragment = re.findall(regex_dep, str(soup2))[0]
    soup_dep = BeautifulSoup(dep_fragment, "html.parser")
    departement = soup_dep.text[1:]

except:
    departement = ""
    
departement

'Polizeidepartement (PD)'

### Schritt 4: Datum rauslesen

In [25]:
soup2.find_all("div")[31].find_all("strong")[1].text

'Interpellation von Markus Schwyn (SVP) und Emil Grabherr (SVP) vom 27.5.1998:'

In [26]:
#Hier ist eingentlich alles drin, auch die Namen

In [27]:
date_junk = soup2.find_all("div")[31].find_all("strong")[1].text
date = date_junk.split(" vom ")

In [28]:
date

['Interpellation von Markus Schwyn (SVP) und Emil Grabherr (SVP)',
 '27.5.1998:']

In [29]:
date = date[1][:-1].strip()

In [30]:
date

'27.5.1998'

### Vorarbeiten 1 für den Loop: Stossen wir auf Fehler?

In [120]:
for file in files: 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("data/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        print (file) #damit finden wir raus, wo es allenfalls Probleme gibt
    
        einreichender1 = soup.find_all("div")[24].find_all("a")[0].text.strip()
            
        #zuerst: wir wollen ja nur jene haben, die überhaupt einen Namen drin haben; 
        #Erkennungsmerkmal: die haben immer 2 Klammern drin
        if "(" and ")" in einreichender1:
            print(einreichender1)
            
#### Yes! So funktioniert das fehlerfrei.

geschaeft2007_496.htm
Walter Angst (AL)

geschaeft1997_538.htm
Christoph Spiess (SD)

geschaeft2015_90.htm
Hans Jörg Käppeli (SP)

geschaeft2000_617.htm
Arthur Bernet (SVP)
Raphaela-Franziska Ulcay-Hauser (SVP)
geschaeft2000_165.htm
Heidi Bucher-Steinegger (Grüne)

geschaeft1998_185.htm
Hans Bachmann (FDP)

geschaeft2002_289.htm
Markus Schwyn (PFZ)
Monika Erfigen (SVP)
geschaeft2000_159.htm
Mauro Tuena (SVP)
Jürg Casparis (SVP)
geschaeft2007_455.htm
Markus Schwyn (PFZ)
Susi Gut (PFZ)
geschaeft2002_262.htm
Bernhard im Oberdorf (SVP)

geschaeft1997_289.htm
Max Fritz (FDP)

geschaeft1999_433.htm
Christian Mettler (SVP)

geschaeft1998_146.htm
Luzi Rüegg (SVP)
Lorenz Habicher (SVP)
geschaeft2000_415.htm
Markus Schwyn (PFZ)
Mauro Tuena (SVP)
geschaeft2000_367.htm
Walter Sutter (SVP)

geschaeft2010_429.htm
Michael Baumer (FDP)
Theresa G. Hensch-Stadelmann (FDP)
geschaeft2006_444.htm
Mirella Wepf (SP)

geschaeft2003_501.htm
Mauro Tuena (SVP)
Roger Liebi (SVP)
geschaeft1998_436.htm
Lorenz Habic

geschaeft2005_44.htm
Susi Gut (PFZ)
Mauro Tuena (SVP)
geschaeft2006_442.htm
Doris Fiala (FDP)

geschaeft2005_378.htm
Luca Jagmetti (FDP)

geschaeft2015_378.htm
Markus Merki (GLP)
Markus Baumann (GLP)
geschaeft2004_157.htm
Doris Fiala (FDP)

geschaeft2005_350.htm
Doris Fiala (FDP)
Christian Aeschbach (FDP)
geschaeft2006_293.htm
Robert Schönbächler (CVP)
Ernst Danner (EVP)
geschaeft2008_401.htm
geschaeft2009_174.htm
Monjek Rosenheim (FDP)
Alexander Jäger (FDP)
geschaeft2011_9.htm
Bernhard im Oberdorf (SVP)
Bruno Wohler (SVP)
geschaeft1998_222.htm
Markus Bischoff (AL)

geschaeft2006_278.htm
Albert Leiser (FDP)

geschaeft2016_278.htm
Reto Vogelbacher (CVP)
Elisabeth Schoch (FDP)
geschaeft2003_467.htm
Bernhard im Oberdorf (SVP)

geschaeft2004_180.htm
Michael Baumer (FDP)
Theresa G. Hensch-Stadelmann (FDP)
geschaeft2001_238.htm
Werner Furrer (SVP)

geschaeft1999_582.htm
Rolf André Siegenthaler-Benz (SVP)

geschaeft2005_146.htm
Susi Gut (PFZ)
Markus Schwyn (PFZ)
geschaeft2004_427.htm
Susi Gut

geschaeft2015_335.htm
Martin Götzl (SVP)
Derek Richter (SVP)
geschaeft2002_188.htm
Cornelia Schaub (SVP)
Mauro Tuena (SVP)
geschaeft2005_490.htm
Markus Schwyn (PFZ)
Roger Liebi (SVP)
geschaeft2013_370.htm
Michael Schmid (FDP)
Tamara Lauber (FDP)
geschaeft2000_39.htm
Lorenz Habicher (SVP)
Marina Garzotto (SVP)
geschaeft1998_253.htm
Oliver B. Meier (SVP)

geschaeft2003_97.htm
Hans-Ulrich Meier (FDP)

geschaeft2014_324.htm
geschaeft2001_507.htm
Raphaela-Franziska Ulcay-Hauser (SVP)
Luzi Rüegg (SVP)
geschaeft2004_330.htm
Bernhard im Oberdorf (SVP)

geschaeft1997_407.htm
Markus Bischoff (AL)
Niklaus Scherr (AL)
geschaeft2004_49.htm
Markus Schwyn (PFZ)
Ruth Anhorn (SVP)
geschaeft2019_18.htm
Katharina Prelicz-Huber (Grüne)
Balz Bürgisser (Grüne)
geschaeft2002_361.htm
Rolf André Siegenthaler-Benz (SVP)
Hans Marolf (SVP)
geschaeft1999_242.htm
Hans Bachmann (FDP)
Monjek Rosenheim (FDP)
geschaeft2002_349.htm
Hans Bachmann (FDP)
Doris Fiala (FDP)
geschaeft1999_530.htm
Marcel Savarioud (SP)

gescha

geschaeft2003_46.htm
Doris Fiala (FDP)
Albert Leiser (FDP)
geschaeft2002_414.htm
Walter Angst (AL)

geschaeft1999_279.htm
Lorenz Habicher (SVP)
Armin Schilter (SVP)
geschaeft1999_245.htm
Ronald Schmid (FDP)
Hans Bachmann (FDP)
geschaeft1999_251.htm
Theo Toggweiler (SVP)

geschaeft2009_472.htm
Peter Stähli-Barth (SP)
Min Li Marti (SP)
geschaeft1997_428.htm
Luzia Vieli-Hardegger (LdU)
Simone Bertogg-Baudet (Grüne)
geschaeft2014_323.htm
geschaeft2004_451.htm
Roger Bartholdi (SVP)
Monika Erfigen (SVP)
geschaeft2004_99.htm
Roger Bartholdi (SVP)
Susi Gut (PFZ)
geschaeft1999_286.htm
Christopher Vohdin (SVP)

geschaeft1999_292.htm
Niklaus Scherr (AL)

geschaeft2003_363.htm
Roger Liebi (SVP)
Monika Erfigen (SVP)
geschaeft1999_55.htm
Luzi Rüegg (SVP)
Emil Grabherr (SVP)
geschaeft2009_32.htm
geschaeft1998_254.htm
Romeo Steiner (CVP)

geschaeft2002_158.htm
Ruth Anhorn (SVP)

geschaeft2000_505.htm
Köbi Möri (Parteilos)

geschaeft2000_511.htm
Monika Piesbergen ()

geschaeft2010_505.htm
Mauro Tuena (

geschaeft2004_40.htm
geschaeft1999_510.htm
Rolf Naef (SP)
Reto Heygel (SP)
geschaeft1999_262.htm
Monika Spring-Gross (SP)
Silvia Biedermann (SP)
geschaeft1999_538.htm
Thomas Marthaler (SP)

geschaeft2003_146.htm
Cornelia Schaub (SVP)
Martin Burger (SVP)
geschaeft1997_427.htm
Myrta Studer (SP)
Esther Maurer (SP)
geschaeft2004_96.htm
Cornelia Schaub (SVP)
Mauro Tuena (SVP)
geschaeft2001_269.htm
Renate Schoch (AL)

geschaeft1999_289.htm
Markus Schwyn (PFZ)

geschaeft2008_120.htm
Mario Mariani (CVP)
Albert Leiser (FDP)
geschaeft2002_396.htm
Beat Badertscher (FDP)
Hans Bachmann (FDP)
geschaeft2004_462.htm
Daniel Leupi (Grüne)

geschaeft1998_273.htm
Hans Diem (CVP)

geschaeft2000_19.htm
Arthur Bernet (SVP)
Monika Erfigen (SVP)
geschaeft2003_350.htm
Marlène Butz (SP)
Werner Sieg (SP)
geschaeft2007_10.htm
Bernhard im Oberdorf (SVP)

geschaeft2002_157.htm
Roger Liebi (SVP)
Walter Isliker (SVP)
geschaeft1999_99.htm
Emil Seliner (SP)
Ueli Keller (SP)
geschaeft2008_478.htm
Claudia Simon (FDP)
Albe

geschaeft2005_111.htm
Roger Liebi (SVP)
Markus Schwyn (PFZ)
geschaeft2002_390.htm
Georg Schmid (CVP)
Robert Schönbächler (CVP)
geschaeft2001_253.htm
Markus Schwyn (PFZ)
Jürg Casparis (SVP)
geschaeft2019_447.htm
Martin Götzl (SVP)
Stephan Iten (SVP)
geschaeft2004_316.htm
Susi Gut (PFZ)

geschaeft2001_509.htm
Kurt Haueter (SVP)
Monika Erfigen (SVP)
geschaeft2009_309.htm
Mirella Wepf (SP)
Claudia Nielsen (SP)
geschaeft2002_151.htm
Roger Liebi (SVP)
Kurt Haueter (SVP)
geschaeft2003_418.htm
Jürg R. Schüepp (FDP)

geschaeft2003_381.htm
Susi Gut (PFZ)
Markus Schwyn (PFZ)
geschaeft2005_313.htm
Markus Schwyn (PFZ)
Mauro Tuena (SVP)
geschaeft2018_330.htm
Yasmine Bourgeois (FDP)
Christian Huser (FDP)
geschaeft2000_530.htm
Pierino Cerliani (Grüne)

geschaeft2009_123.htm
Hans Jörg Käppeli (SP)
Joe A. Manser (SP)
geschaeft2004_672.htm
Monika Erfigen (SVP)
Roger Liebi (SVP)
geschaeft2016_401.htm
Katharina Prelicz-Huber (Grüne)

geschaeft2013_236.htm
Bernhard im Oberdorf (SVP)
Roger Bartholdi (SVP)
ge

geschaeft2002_445.htm
Hans-Ulrich Meier (FDP)
Alexander Weber (SVP)
geschaeft1999_200.htm
Marcel Knörr (FDP)

geschaeft2003_119.htm
Kurt Maeder (CVP)

geschaeft2019_393.htm
Elisabeth Schoch (FDP)
Martina Zürcher (FDP)
geschaeft2002_322.htm
Beat Badertscher (FDP)
Monjek Rosenheim (FDP)
geschaeft2017_267.htm
Anjushka Früh (SP)
Pawel Silberring (SP)
geschaeft2002_450.htm
Markus Schwyn (PFZ)
Raphaela-Franziska Ulcay-Hauser (SVP)
geschaeft2001_593.htm
Hans Bachmann (FDP)

geschaeft2010_80.htm
Severin Pflüger (FDP)
Claudia Simon (FDP)
geschaeft1997_450.htm
Alfred Heer (SVP)
Thomas Meier (SVP)
geschaeft2001_544.htm
Balthasar Glättli (Grüne)

geschaeft2001_236.htm
Rolf André Siegenthaler-Benz (SVP)
Monika Erfigen (SVP)
geschaeft2004_401.htm
Susi Gut (PFZ)

geschaeft2019_98.htm
Markus Merki (GLP)
Stefan Urech (SVP)
geschaeft2004_367.htm
Ernst Danner (EVP)
Hanspeter Kunz (EVP)
geschaeft1999_39.htm
Jeannette Linggi Werner (SP)
Dominik Schaub (SP)
geschaeft2000_46.htm
Werner Furrer (SVP)
Bruno Sid

geschaeft2000_234.htm
Rolf André Siegenthaler-Benz (SVP)
Bruno Sidler (SVP)
geschaeft2004_610.htm
Bruno Sidler (SVP)
Theo Hauri (SVP)
geschaeft2004_176.htm
Catrina Luchsinger Gähwiler (FDP)

geschaeft2003_491.htm
Bernhard im Oberdorf (SVP)

geschaeft2016_259.htm
geschaeft2009_71.htm
Monjek Rosenheim (FDP)

geschaeft2000_591.htm
Theo Hauri (SVP)
Peter Mächler (SVP)
geschaeft2000_8.htm
Rolf André Siegenthaler-Benz (SVP)
Jürg Casparis (SVP)
geschaeft2004_412.htm
Cornelia Schaub (SVP)
Bernhard im Oberdorf (SVP)
geschaeft2015_173.htm
geschaeft2004_406.htm
Markus Knauss (Grüne)
Niklaus Scherr (AL)
geschaeft2001_231.htm
Mauro Tuena (SVP)
Jürg Casparis (SVP)
geschaeft2002_319.htm
Roger Liebi (SVP)
Mauro Tuena (SVP)
geschaeft2002_325.htm
Doris Fiala (FDP)

geschaeft2003_10.htm
Cornelia Schaub (SVP)
Markus Schwyn (PFZ)
geschaeft2003_11.htm
Bernhard im Oberdorf (SVP)

geschaeft2019_49.htm
Roger Bartholdi (SVP)
Dubravko Sinovcic (SVP)
geschaeft2002_318.htm
Roger Liebi (SVP)
Mauro Tuena (SVP)
gesch

### Vorarbeiten 2 für den Loop: Nur Geschäfte mit mehreren Einreichenden rausfiltern:

In [136]:
files = os.listdir("motionen")

for file in files: 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("motionen/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #So filtern wir nur jene raus, die mehrere Einreichende haben:
        einreichende = soup.find_all("div")[24].find_all("a")
        if len(einreichende) > 1: 
             
            #print (file) #damit finden wir raus, wo es allenfalls Probleme gibt
            
            #Jetzt filtern wir jene, die überhaupt einen Namen drin haben; Erkennungsmerkmal: Klammern
            
            einreichender1 = einreichende[0].text.strip()
            if "(" and ")" in einreichender1:
                #print(einreichender1)
                
                #try:
                    #einreichender2 = einreichende[1].text.strip()
                    #print(einreichender2)
                
                #except: 
                    #einreichender2 = ""
                        
                try:
                    einreichender3 = einreichende[2].text.strip()
                    print(einreichender3)
                except: 
                    einreichender3 = ""

So haben wir rausgefunden: Es gibt keine Geschäfte, die von mehr als 2 Personen eingereicht wurden – das vereinfacht die Sache wesentlich: Wir können also eine Liste machen, die durchs Band zwei Personen umfasst.

### Vorarbeiten 3 für den Loop: Jetzt holen wir alle gesuchten Daten raus

In [354]:
from tqdm import tqdm

In [355]:
files = os.listdir("alle_geschaefte")
df_list = []

for file in tqdm(files): 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("alle_geschaefte/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #Jetzt filtern wir nur jene raus, die mehrere Einreichende haben:
        einreichende = soup.find_all("div")[24].find_all("a")
        if len(einreichende) > 1: 
            
            #Jetzt filtern wir jene, die überhaupt einen Namen drin haben; Erkennungsmerkmal: Klammern
            einreichender1 = einreichende[0].text.strip()
            if "(" and ")" in einreichender1:
                
                name1 = einreichender1.split(" (")[0]
                partei1 = einreichender1.split(" (")[1][:-1]
                
                einreichender2 = einreichende[1].text.strip()
                name2 = einreichender2.split(" (")[0]
                partei2 = einreichender2.split(" (")[1][:-1]
                
                geschaeft_nr = soup.find_all("div")[22].find_all("span")[1].text
                #datum = soup.find_all("div")[31].find_all("strong")[1].text.split(" vom ")[1][:-1].strip()
                try: 
                    departement = soup.find_all("div")[28].find_all("span")[2].text
                except: 
                    departement = ""
                            
                #Zum Schluss: Wir wollen nur jene in unserem Dict mit unterschiedlichen Parteien
                if partei1 != partei2:
                    mini_dict = {
                        #"Datum" : datum,
                        "Geschäft" : geschaeft_nr,
                        "Einreichender 1" : name1,
                        "Partei 1" : partei1,
                        "Einreichender 2" : name2,
                        "Partei 2" : partei2,
                        "Departement" : departement
                    }
                    df_list.append(mini_dict)  



  0%|          | 0/5086 [00:00<?, ?it/s]

  0%|          | 2/5086 [00:00<04:41, 18.09it/s]

  0%|          | 5/5086 [00:00<04:10, 20.27it/s]

  0%|          | 8/5086 [00:00<03:49, 22.16it/s]

  0%|          | 12/5086 [00:00<03:26, 24.59it/s]

  0%|          | 15/5086 [00:00<03:25, 24.64it/s]

  0%|          | 18/5086 [00:00<03:25, 24.66it/s]

  0%|          | 21/5086 [00:01<06:11, 13.64it/s]

  0%|          | 23/5086 [00:01<05:46, 14.60it/s]

  1%|          | 26/5086 [00:01<05:01, 16.79it/s]

  1%|          | 29/5086 [00:01<04:32, 18.55it/s]

  1%|          | 32/5086 [00:01<04:20, 19.41it/s]

  1%|          | 35/5086 [00:01<03:59, 21.12it/s]

  1%|          | 39/5086 [00:01<03:31, 23.81it/s]

  1%|          | 42/5086 [00:01<03:23, 24.83it/s]

  1%|          | 45/5086 [00:02<03:28, 24.14it/s]

  1%|          | 48/5086 [00:02<03:16, 25.60it/s]

  1%|          | 51/5086 [00:02<03:11, 26.31it/s]

  1%|          | 55/5086 [00:02<03:00, 27.82it/s]

  1%|          | 58/5086 [00:02<03:00, 27

KeyboardInterrupt: 

Beim Datum und Departement funktioniert was nicht, sonst läufts!

### >>> Mit einfachem Loop testen, wo Datum rauslesen nicht funktioniert:

In [356]:
files = os.listdir("alle_geschaefte")

for file in files: 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("alle_geschaefte/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #So filtern wir nur jene raus, die mehrere Einreichende haben:
        einreichende = soup.find_all("div")[24].find_all("a")
        if len(einreichende) > 1: 
             
            print(file)
            
            try:
                datum = soup.find_all("div")[31].find_all("strong")[0].text.split(" vom ")[1][:-1].strip()
            except:
                try: 
                    datum = soup.find_all("div")[31].find_all("strong")[1].text.split(" vom ")[1][:-1].strip()
                except:
                    datum = soup.find_all("div")[31].find_all("td")[3].find_all("p")[1].text.split(" vom ")[1][:-1].strip()  
            
            print(datum)

geschaeft2001_362.htm
27.6.2001
geschaeft2000_171.htm
5.4.2000
geschaeft2012_99.htm
14.03.2012
geschaeft2014_227.htm
02.07.2014
geschaeft2011_376.htm


IndexError: list index out of range



 15%|█▌        | 778/5086 [00:45<02:45, 26.07it/s]

In [72]:
# Jetzt testen wir die fehlerhaften File:
file = open("alle_geschaefte/geschaeft2012_99.htm", "r")
text = file.read()
soup = BeautifulSoup(text, "html.parser")

#Erkenntnis: Das Datim ist manchmal nicht in folgendem Fragment:
#soup.find_all("div")[31].find_all("strong")[1].text
#sondern in diesem hier:
#soup.find_all("div")[31].find_all("strong")[0].text
#Fazit: es braucht hier eine try-funktion

#manchmal findet man es so: (Ist das eventuell der besser Code für alle? > testen!)
soup.find_all("div")[31].find_all("td")[3].find_all("p")[1].text.split(" vom ")[1][:-1].strip()  

# Fazit: Geht, wird aber zu mühsam. Ich versuche es stttdessen mir Regex in den Griff zu bekommen

'14.03.2012'

In [362]:
import re

file = open("alle_geschaefte/geschaeft2004_406.htm", "r")
text = file.read()
soup = BeautifulSoup(text, "html.parser")

#wir mussten hier nachträglich noch eine Wihtespace-Putzfunktion einbauen, weils vor dem Jahr manchmal einen hat
regex = r"((?<=\svom\s)\d*\.\d*\.\s*\d*)"
datum = re.findall(regex, str(soup))[0].replace(" ", "")
datum

'18.8.2004'

In [363]:
#Das funktioniert, darum testen wir es jetzt im Loop mit allen files:

files = os.listdir("alle_geschaefte")

for file in files: 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("alle_geschaefte/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #So filtern wir nur jene raus, die mehrere Einreichende haben:
        einreichende = soup.find_all("div")[24].find_all("a")
        if len(einreichende) > 1: 
             
            print(file)
            
            regex = r"((?<=\svom\s)\d*\.\d*\.\s*\d*)"
            datum = re.findall(regex, str(soup))[0].replace(" ", "")

            print(datum)

geschaeft2001_362.htm
27.6.2001
geschaeft2000_171.htm
5.4.2000
geschaeft2012_99.htm
14.03.2012
geschaeft2014_227.htm
02.07.2014
geschaeft2011_376.htm
05.10.2011
geschaeft2009_562.htm
25.11.2009
geschaeft2000_617.htm
20.12.2000
geschaeft2009_238.htm
03.06.2009
geschaeft2002_289.htm
21.8.2002
geschaeft2000_159.htm
29.3.2000
geschaeft2017_327.htm
20.09.2017
geschaeft2016_134.htm
20.04.2016
geschaeft2007_455.htm
22.8.2007
geschaeft2012_262.htm
20.06.2012
geschaeft2006_134.htm
3.5.2006
geschaeft2002_276.htm
21.8.2002
geschaeft2008_16.htm
26.05.2010
geschaeft2012_66.htm
29.02.2012
geschaeft1998_146.htm
13.5.1998
geschaeft2000_415.htm
30.8.2000
geschaeft2004_3.htm
7.1.2004
geschaeft2006_487.htm
08.11.2006
geschaeft2005_550.htm
16.05.2013
geschaeft2010_415.htm
29.09.2010
geschaeft2011_160.htm
18.05.2011
geschaeft2010_429.htm
06.10.2010
geschaeft2005_42.htm
9.2.2005
geschaeft2019_6.htm
09.01.2019
geschaeft2007_643.htm
5.12.2007
geschaeft2003_501.htm
17.12.2003
geschaeft1998_344.htm
28.10.1998
g

KeyboardInterrupt: 

### Heureka! Also: Zweiter Anlauf kompletter Loop:

In [1]:
import os
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re

files = os.listdir("alle_geschaefte")
df_list = []

for file in tqdm(files): 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("alle_geschaefte/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #Jetzt filtern wir nur jene raus, die mehrere Einreichende haben:
        einreichende = soup.find_all("div")[24].find_all("a")
        if len(einreichende) > 1: 
            
            #Jetzt filtern wir jene, die überhaupt einen Namen drin haben; Erkennungsmerkmal: Klammern
            einreichender1 = einreichende[0].text.strip()
            if "(" and ")" in einreichender1:
                
                name1 = einreichender1.split(" (")[0]
                partei1 = einreichender1.split(" (")[1][:-1]
                
                einreichender2 = einreichende[1].text.strip()
                name2 = einreichender2.split(" (")[0]
                partei2 = einreichender2.split(" (")[1][:-1]
                
                geschaeft_nr = soup.find_all("div")[22].find_all("span")[1].text
                
                regex = r"((?<=\svom\s)\d*\.\d*\.\s*\d*)"
                datum = re.findall(regex, str(soup))[0].replace(" ", "")
                
                try: 
                    regex_dep = r"((?<=Zuständiges Departement).*\n*.*)"
                    dep_fragment = re.findall(regex_dep, str(soup))[0]
                    soup_dep = BeautifulSoup(dep_fragment, "html.parser")
                    departement = soup_dep.text[1:]

                except: 
                    departement = ""
                            
                #Zum Schluss: Wir wollen nur jene in unserem Dict mit unterschiedlichen Parteien
                if partei1 != partei2:
                    mini_dict = {
                        "Datum" : datum,
                        "Geschäft" : geschaeft_nr,
                        "Einreichender 1" : name1,
                        "Partei 1" : partei1,
                        "Einreichender 2" : name2,
                        "Partei 2" : partei2,
                        "Departement" : departement
                    }
                    df_list.append(mini_dict)  

100%|██████████| 7026/7026 [05:20<00:00, 21.91it/s]


In [2]:
len(df_list)

1196

In [4]:
df = pd.DataFrame(df_list)
df.head(10)

,Datum,Geschäft,Einreichender 1,Partei 1,Einreichender 2,Partei 2,Departement
0,25.11.2009,2009/562,Kurt Hüssy,SVP,Peter Anderegg,EVP,Polizeidepartement (PD)
1,08.01.2020,2020/2,Marcel Bührig,Grüne,Natascha Wey,SP,Präsidialdepartement (PRD)
2,28.09.2011,2011/362,Alecs Recher,AL,Maleica Landolt,GLP,Finanzdepartement (FD)
3,20.10.2004,2004/541,Niklaus Scherr,AL,Kurt Maeder,CVP,Hochbaudepartement (HBD)
4,29.01.20,1998/191,Rolf Walther,FDP,Jürg Liebermann,,Finanzdepartement (FD)
5,21.8.2002,2002/289,Markus Schwyn,PFZ,Monika Erfigen,SVP,Hochbaudepartement (HBD)
6,23.11.2011,2011/438,Linda Bär,SP,Matthias Probst,Grüne,Präsidialdepartement (PRD)
7,3.5.2006,2006/134,Richard Rabelbauer,EVP,Robert Schönbächler,CVP,Departement der Industriellen Betriebe (DIB)
8,26.05.2010,2008/16,Rebekka Wyler,SP,Pierino Cerliani,Grüne,Finanzdepartement (FD)
9,29.8.2007,2007/469,Peter Anderegg,EVP,Daniel Meier,CVP,Polizeidepartement (PD)


In [367]:
df.to_csv("Dataframe_komplett.csv")

### Kontrolle und Datenbereinigung

In [5]:
df["Partei 1"].value_counts().head(20)

SP           283
Grüne        204
FDP          130
PFZ          112
AL            99
SVP           92
CVP           91
              57
GLP           54
EVP           40
Parteilos     14
CSP           11
SL             3
FraP           3
32 KB          1
LdU            1
85 KB          1
Name: Partei 1, dtype: int64

In [6]:
# Wer sind diese Leute, die nicht als Parteilos vermerkt sind, sondern gar keine haben?
df[df["Partei 1"] == ""]["Einreichender 1"].value_counts()

Gregor Bucher          18
Andreas Ammann         10
Andreas J. Schlegel     6
Urs Rechsteiner         6
Peter Marti             4
Jürg Liebermann         4
Fakir Atalay            4
Placidus Maissen        2
Anton Stäbler           2
Salomon Browar          1
Name: Einreichender 1, dtype: int64

In [7]:
# Jetzt machen wir zum Reaprieren ein Dataframe, in dem wir den Namen die Parteien zuordnen:

Reparatur_dict = {"Gregor Bucher":"Grüne",
                  "Andreas Ammann":"SP",
                  "Urs Rechsteiner":"CVP",
                  "Andreas J. Schlegel":"FDP",
                  "Jürg Liebermann":"FDP",
                  "Peter Marti":"FDP",
                  "Fakir Atalay":"SP",
                  "Anton Stäbler":"CVP",
                  "Placidus Maissen":"CVP",
                  "Salomon Browar":"Grüne",
                  "Monika Piesbergen":"FDP",
                  "Hanna Lienhard":"FDP",
                  "Urs Lauffer":"FDP",
                  "Esther Ponti-Weder":"CVP"}

In [8]:
#Einer von vielen misslungenen Ansätzen:
def Reparatur(Person):
    if Person in Reparatur_dict:
        return Reparatur_dict[Person]

 df["Partei 1"] = df["Einreichender 1"].apply(Reparatur)
#geht nicht, weil es so die Werte aller anderen mit none überschreibt

IndentationError: unexpected indent (<ipython-input-8-11c97e9abe3e>, line 2)

In [371]:
#So steuern wir eine leere Partei an, deren Position wir kennen:
df.loc[41, 'Einreichender 1']

'Jürg Liebermann'

In [10]:
#Jetzt machen wir eine Liste, in der der Index aller fehlerhaften Positionen für den Einreichenden 1 enthalten sind:
fehlerposition1 = df[df["Partei 1"] == ""]["Einreichender 1"].index.tolist()

In [11]:
def Reparatur(Person):
    return Reparatur_dict[Person]

#Hier steuern wir nn für jede fehlerhafte Zeile die (leere) Partei-Zelle an, und füllen diese mit dem richtigen Wert.
#Dazu ersetzen wir die Leerstelle "" mit dem korekten Wert.
#Diesen erhalten wir, indem wir den Namen der gleichen Zeile in unsere Funktion schicken, 
#die aus dem Reparatur-Dict die zugehörige Partei holt.
for zeile in fehlerposition1:
    df.loc[zeile, 'Partei 1'] = df.loc[zeile, 'Partei 1'].replace("", Reparatur(df.loc[zeile, "Einreichender 1"]))

In [12]:
# Jetzt testen wir das – Heureka!
df[df["Einreichender 1"] == "Gregor Bucher"]

,Datum,Geschäft,Einreichender 1,Partei 1,Einreichender 2,Partei 2,Departement
108,18.5.2005,2005/194,Gregor Bucher,Grüne,Daniel Leupi,Grüne,Hochbaudepartement (HBD)
164,15.11.2006,2006/495,Gregor Bucher,Grüne,Ernst Danner,EVP,Präsidialdepartement (PRD)
203,18.5.2005,2005/190,Gregor Bucher,Grüne,Muriel Herzig,Grüne,Sozialdepartement (SD)
267,18.5.2005,2005/192,Gregor Bucher,Grüne,Eva Virag Jansen,Grüne,Hochbaudepartement (HBD)
271,18.5.2005,2005/193,Gregor Bucher,Grüne,Pierino Cerliani,Grüne,Hochbaudepartement (HBD)
316,08.01.20,2007/344,Gregor Bucher,Grüne,Ernst Danner,EVP,Präsidialdepartement (PRD)
355,1.9.2004,2004/457,Gregor Bucher,Grüne,Daniel Leupi,Grüne,Schul- und Sportdepartement (SSD)
550,16.6.2004,2004/321,Gregor Bucher,Grüne,Walter Angst,AL,Schul- und Sportdepartement (SSD)
612,08.01.20,2006/406,Gregor Bucher,Grüne,Muriel Herzig,Grüne,Sozialdepartement (SD)
726,22.9.2004,2004/515,Gregor Bucher,Grüne,Pierino Cerliani,Grüne,Präsidialdepartement (PRD)


In [13]:
df["Partei 1"].value_counts()

SP           297
Grüne        223
FDP          144
PFZ          112
CVP          101
AL            99
SVP           92
GLP           54
EVP           40
Parteilos     14
CSP           11
FraP           3
SL             3
85 KB          1
32 KB          1
LdU            1
Name: Partei 1, dtype: int64

In [14]:
#Weils geklappt hat,machen wir hier das gleiche noch für die Einreichenden 2:
def Reparatur(Person):
    return Reparatur_dict[Person]

fehlerposition2 = df[df["Partei 2"] == ""]["Einreichender 2"].index.tolist()

for zeile in fehlerposition2:
    df.loc[zeile, 'Partei 2'] = df.loc[zeile, 'Partei 2'].replace("", Reparatur(df.loc[zeile, "Einreichender 2"]))

Achtung, durch die Reparatur sind möglicherweise Paare der gleichen Partei oder des gleichen Blocks entstanden, die müssen wir rausfiltern:

In [15]:
#Hier ist die Variante für Parteien:
for zeile in range(0, len(df)-1):
    if df.loc[zeile, "Partei 1"] == df.loc[zeile, "Partei 2"]:
        df.drop(zeile, inplace = True)

In [33]:
len(df)

1115

In [17]:
#Weil wir endlich am Ziel sind, speichern wir den bereinigten Dataframe:
df.to_csv("Dataframe_bereinigt_parteiuebergreifend.csv")

### Hier werten wir die Zahlen nach Legislatur aus (davor die Daten säubern)

bei diesen Positionen stimmt beim Datum etwas Grundsätzliches nicht, weshalb wir es mit einem annäherungsweisen Datum aufgrund der Geschäftnummer ersetzen:

In [35]:
fehlerliste = ["29.01.20","08.01.20","18.12.19"]

for fehler in fehlerliste:
    fehlerposition = df[df["Datum"] == fehler].index.tolist()
    for zeile in fehlerposition:
        Jahr = df.loc[zeile, "Geschäft"].split("/")[0]
        Ersatzdatum = "31.12." + Jahr
        df.loc[zeile, "Datum"] = df.loc[zeile, "Datum"].replace(fehler, Ersatzdatum)

Hier korrigieren wir zwei weitere Fehler, bei denen die Daten zwar stimmen, aber falsch formatiert sind (Leerschläge, zweistellige Jahreszahlen):

In [36]:
df.head(5)

,Datum,Geschäft,Einreichender 1,Partei 1,Einreichender 2,Partei 2,Departement
0,25.11.2009,2009/562,Kurt Hüssy,SVP,Peter Anderegg,EVP,Polizeidepartement (PD)
1,08.01.2020,2020/2,Marcel Bührig,Grüne,Natascha Wey,SP,Präsidialdepartement (PRD)
2,28.09.2011,2011/362,Alecs Recher,AL,Maleica Landolt,GLP,Finanzdepartement (FD)
3,20.10.2004,2004/541,Niklaus Scherr,AL,Kurt Maeder,CVP,Hochbaudepartement (HBD)
5,21.8.2002,2002/289,Markus Schwyn,PFZ,Monika Erfigen,SVP,Hochbaudepartement (HBD)


In [37]:
len(df)

1115

In [54]:
df_laenge = len(df)
for zeile in range(0, df_laenge-1):
    
    #hier holen wir jene Daten raus, die nur eine zweistellige als Jahreszahl haben und vervollständigen sie:
    if df.loc[zeile,"Datum"][-3:-2] == ".":
        if df.loc[zeile,"Datum"][-2:] >= "80":
            df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"][:-2]+"19"+df.loc[zeile,"Datum"][-2:]
        if df.loc[zeile,"Datum"][-2:] <= "21":
            df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"][:-2]+"20"+df.loc[zeile,"Datum"][-2:] 
    
    #hier holen wir jene Daten raus, die einen Leerschlag drin haben und löschen ihn raus
    if df.loc[zeile,"Datum"][-3:-2] == " ":
        df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"][:-3]+df.loc[zeile,"Datum"][-2:]
        
    #hier holen wir jene Daten mit vierer-Jahreszahlen raus, die einen Leerschlag davor haben und löschen ihn raus
    if df.loc[zeile,"Datum"][-5:-4] == " ":
        df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"][:-5]+df.loc[zeile,"Datum"][-4:]
    
    #Hier holen wir jene raus, die zu kurz sind, weil die Jahreszahl ganz fehlt:
    if len(df.loc[zeile,"Datum"]) == 6:
        Jahr = df.loc[zeile, "Geschäft"].split("/")[0]
        df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"]+Jahr
        
        print(df.loc[zeile,"Datum"])

KeyError: 4

In [55]:
df["Datum"] = pd.to_datetime(df['Datum'], format='%d.%m.%Y')
df = df.set_index("Datum")
df.head(5)

,Geschäft,Einreichender 1,Partei 1,Einreichender 2,Partei 2,Departement
Datum,,,,,,
2009-11-25,2009/562,Kurt Hüssy,SVP,Peter Anderegg,EVP,Polizeidepartement (PD)
2020-01-08,2020/2,Marcel Bührig,Grüne,Natascha Wey,SP,Präsidialdepartement (PRD)
2011-09-28,2011/362,Alecs Recher,AL,Maleica Landolt,GLP,Finanzdepartement (FD)
2004-10-20,2004/541,Niklaus Scherr,AL,Kurt Maeder,CVP,Hochbaudepartement (HBD)
2002-08-21,2002/289,Markus Schwyn,PFZ,Monika Erfigen,SVP,Hochbaudepartement (HBD)


Jetzt messen wir für die Statistik die Zahl pro Legislatur:

In [56]:
df_1990 = df["1990-05-15":"1994-05-14"]
df_1994 = df["1994-05-15":"1998-05-14"]
df_1998 = df["1998-05-15":"2002-05-14"]
df_2002 = df["2002-05-15":"2006-05-14"]
df_2006 = df["2006-05-15":"2010-05-14"]
df_2010 = df["2010-05-15":"2014-05-14"]
df_2014 = df["2014-05-15":"2018-05-14"]
df_2018 = df["2018-05-15":"2022-05-14"]

print("1990-94: "+str(len(df_1990)))
print("1994-98: "+str(len(df_1994)))
print("1998-02: "+str(len(df_1998)))
print("2002-06: "+str(len(df_2002)))
print("2006-10: "+str(len(df_2006)))
print("2010-14: "+str(len(df_2010)))
print("2014-18: "+str(len(df_2014)))
print("2018-22: "+str(len(df_2018)))

1990-94: 3
1994-98: 16
1998-02: 133
2002-06: 242
2006-10: 233
2010-14: 162
2014-18: 197
2018-22: 129
